In [ ]:
from tensorflow.keras import datasets

In [ ]:
import os
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, MaxPooling2D
from tensorflow.keras import datasets
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras import Input
from tensorflow.keras.layers import Dropout, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
batch_size = 128
img_height = 128
img_width = 128

In [ ]:
dataset_path = '/content/drive/MyDrive/ZB/DL_project_2/temp_data/Training'
train_x = tf.keras.utils.image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset='training',
    seed=123,
    image_size=(img_height,img_width),
    batch_size=batch_size
)

Found 84914 files belonging to 5 classes.
Using 67932 files for training.


In [ ]:
val_x = tf.keras.utils.image_dataset_from_directory(
    dataset_path,validation_split=0.2,
    subset='validation',
    seed=123,
    image_size=(img_height,img_width),
    batch_size=batch_size
)

Found 84914 files belonging to 5 classes.
Using 16982 files for validation.


In [ ]:
base_model = ResNet50(include_top=False, pooling = 'avg' , input_shape = (128 ,128 ,3), weights = 'imagenet')
base_model.trainable = False

94765736/94765736 [==============================] - 4s 0us/step


In [ ]:
print(len(base_model.layers))

176


In [ ]:
for i in base_model.layers[149:]:
    i.trainable = True

In [ ]:
inputs = Input(shape=(128,128,3))
x = tf.keras.applications.resnet50.preprocess_input(inputs)
x = base_model(x)
#x = base_model(x, training = False)
x = Flatten()(x)		
outputs = Dense(5, activation = 'softmax')(x)	
model_res = tf.keras.Model(inputs, outputs)	

In [ ]:
model_res.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001), loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
early = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 10)
model_res.fit(
  train_x,
  validation_data=val_x,
  epochs=5,
  callbacks = [early])

Epoch 1/5
531/531 [==============================] - 12671s 24s/step - loss: 1.0078 - accuracy: 0.5989 - val_loss: 0.8346 - val_accuracy: 0.6617
Epoch 2/5
531/531 [==============================] - 175s 325ms/step - loss: 0.8102 - accuracy: 0.6707 - val_loss: 0.7847 - val_accuracy: 0.6760
Epoch 3/5
531/531 [==============================] - 171s 319ms/step - loss: 0.7509 - accuracy: 0.6936 - val_loss: 0.7312 - val_accuracy: 0.7013
Epoch 4/5
531/531 [==============================] - 171s 318ms/step - loss: 0.7184 - accuracy: 0.7063 - val_loss: 0.7450 - val_accuracy: 0.6936
Epoch 5/5
531/531 [==============================] - 171s 318ms/step - loss: 0.6958 - accuracy: 0.7158 - val_loss: 0.7224 - val_accuracy: 0.7044


In [ ]:
model_res.save("resnet50_transfer_learning_practice_128X128.h5")